In [1]:
import pyvista as pv
# import topogenesis  as tp
import numpy as np
from honeybee_plus.hbsurface import HBSurface
from honeybee_plus.radiance.recipe.solaraccess.gridbased import SolarAccessGridBased
from honeybee_plus.radiance.analysisgrid import AnalysisGrid
import os

cwd = os.getcwd()
src = os.path.dirname(cwd)

Path to radiance is set to: /usr/local/radiance
Path to perl is set to: /usr/bin


In [2]:
filepath = os.path.join(src, "data", "test_room_01.obj")
mesh = pv.read(filepath)
# print(mesh.is_all_triangles())

# srf_dict = {
#     'name': 'floor', 
#     'vertices': [[(0, 0, 3.2), (-4.242640687119285, 4.242640687119286, 3.2), (-1.2727922061357848, 7.212489168102785, 3.2), (2.9698484809835, 2.9698484809834995, 3.2)]], 
#     'surface_material': {
#         'modifier': 'void', 
#         'type': 'plastic', 
#         'name': 'generic_floor', 
#         'r_reflectance': 0.2, 
#         'g_reflectance': 0.2, 
#         'b_reflectance': 0.2, 
#         'specularity': 0.0, 
#         'roughness': 0.0}, 
#     'surface_type': 2.0
# }
face_list = list(mesh.faces)
hb_surfaces = []
e = 0
while e < len(face_list):
    v_count = face_list[e]
    vertices = []
    for v in range(v_count):
        e += 1
        vertices.append(list(mesh.points[face_list[e]]))
    srf_dict = {
        "name": "",
        "vertices": vertices, # [[(x, y, z), (x1, y1, z1), (x2, y2, z2)]],
        "surface_material": {
            'modifier': 'void', 
            'type': 'plastic', 
            'name': 'generic_floor', 
            'r_reflectance': 0.2, 
            'g_reflectance': 0.2, 
            'b_reflectance': 0.2, 
            'specularity': 0.0, 
            'roughness': 0.0},
        "surface_type": 0  # 0: wall, 5: window
        }
    hbsrf = HBSurface.from_json(srf_dict)
    hb_surfaces.append(hbsrf)
    e += 1


print(hb_surfaces)
# print(type(mesh))
# print(mesh.points)
print(mesh.faces)

[HBSurface::a4d03c69-1176::Wall, HBSurface::81205123-8641::Wall, HBSurface::2c6865d1-8163::Wall, HBSurface::03d25282-64f4::Wall, HBSurface::19316733-5c47::Wall, HBSurface::36795e31-7bf1::Wall, HBSurface::296f7037-d357::Wall, HBSurface::048afc44-c559::Wall, HBSurface::20dea317-e433::Wall, HBSurface::785f1a27-eba7::Wall, HBSurface::e70518ff-7c7a::Wall, HBSurface::fa5eaecb-1b5b::Wall, HBSurface::4bd1cd7d-67d0::Wall, HBSurface::8eec49db-60a3::Wall, HBSurface::c6876e77-d27c::Wall, HBSurface::d4934a3a-3f84::Wall, HBSurface::29f12e66-4491::Wall, HBSurface::edf908c2-ba91::Wall, HBSurface::cf598c2e-b19e::Wall, HBSurface::a39f5e6a-9344::Wall, HBSurface::b8847307-8d06::Wall, HBSurface::cd578b79-b024::Wall, HBSurface::7afa2117-889f::Wall, HBSurface::69d61426-44f5::Wall, HBSurface::fdc407a3-f445::Wall, HBSurface::cbfff63e-87d6::Wall, HBSurface::87a6340b-5380::Wall, HBSurface::c6324a95-1e79::Wall, HBSurface::8f78321a-dc2e::Wall, HBSurface::3f1e8317-3376::Wall, HBSurface::2eb56edf-b42d::Wall, HBSurfa

In [3]:
# # generate grid of test points
# analysis_grid = room.generate_test_points(grid_size=0.5, height=0.75)
test_pts = [(0, 0, 0), (1, 1, 0), (3, 2, 0)]
test_vec = [(0, 0, 1), (0, 0, 1), (0, 0, 1)]
analysis_grid = AnalysisGrid.from_points_and_vectors(test_pts, test_vec, 'test_grid')

# define sun vectors
sun_vectors = [
            (-0.810513, 0.579652, -0.084093), (-0.67166, 0.702357, -0.235729),
            (-0.487065, 0.798284, -0.354275), (-0.269301, 0.8609, -0.431657),
            (-0.033196, 0.885943, -0.462605), (0.20517, 0.871705, -0.445013),
            (0.429563, 0.819156, -0.380077), (0.624703, 0.731875, -0.272221),
            (0.777301, 0.615806, -0.128788)]

# define sun hours : A list of hours of the year for each sun vector
# there are 87600 hours in a year, so the following integers refer to specific hours throughout the year
hoys = [1908, 2000, 2530, 3254, 3658, 4000, 4116, 6324, 8508]

# # put the recipe together
# rp = GridBased(sky=sky, analysis_grids=(analysis_grid,), simulation_type=0, hb_objects=(room,))
rp = SolarAccessGridBased(sun_vectors, hoys, [analysis_grid], hb_objects=hb_surfaces)

Found 78 opaque surfaces.
Found 0 fenestration surfaces.
Found 0 window-groups.


In [4]:
# write simulation to folder
batch_file = rp.write(
    target_folder='.', project_name='external_room_sun')

Writing recipe contents to: ./external_room_sun/solaraccess


In [5]:
# run the simulation
for com in rp._commands:
    print(com)
rp.run(batch_file, debug=False)

# https://www.radiance-online.org/learning/documentation/manual-pages/pdfs/rmtxop.pdf
# Eleonora: rmtxop multiplies the daylight coefficient matrices with the sky vectors. This is necessary for getting illuminance from daylight coefficients computed w.r.t. rgb components. 
# Sun Access of points, and how to interpret them https://floyd.lbl.gov/radiance/refer/usman2.pdf

#!/usr/bin/env bash



cd ./external_room_sun/solaraccess

/usr/local/radiance/bin/oconv -f scene/opaque/external_room_sun..opq.mat scene/opaque/external_room_sun..opq.rad scene/glazing/external_room_sun..glz.mat scene/glazing/external_room_sun..glz.rad sky/analemma.rad > external_room_sun.oct
/usr/local/radiance/bin/rcontrib -aa 0.0 -ab 0 -ad 512 -ar 16 -as 128 -dc 1.0 -dj 0.0 -dp 64 -ds 0.5 -dr 0 -dt 0.0 -I -lr 4 -lw 0.05 -M ./sky/analemma.mod -ss 0.0 -st 0.85 -y 3 external_room_sun.oct < external_room_sun.pts > result/external_room_sun.dc
/usr/local/radiance/bin/rmtxop -c 47.4 119.9 11.6 -fa result/external_room_sun.dc > result/external_room_sun.ill


True

In [6]:
# results - in this case it will be an analysis grid
result = rp.results()[0]
print(rp.results()[0])

Unloading the current values from the analysis grids.
Unloading the current values from the analysis grids.
0.000 0.000 0.000 0.000 0.000 1.000
1.000 1.000 0.000 0.000 0.000 1.000
3.000 2.000 0.000 0.000 0.000 1.000


In [7]:
# Plot using the ITKplotter
pl = pv.PlotterITK()
pl.add_mesh(mesh, smooth_shading=True)
pl.show(True)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…